In [ ]:
!pip3 install feathrs

In [ ]:
import logging
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.DEBUG)

In [1]:
from feathrs import *
import os
os.environ['AZURE_RESOURCE_GROUP']='MyTestResourceGroup'
os.environ['AZURE_CLIENT_ID']='fb0fa1dc-9db0-4ad9-9c65-85ce0f9a31ab'
os.environ['AZURE_TENANT_ID']='72f988bf-86f1-41af-91ab-2d7cd011db47'
os.environ['AZURE_CLIENT_SECRET']='YOUR_AZURE_CLIENT_SECRET'
os.environ['SYNAPSE_DEV_URL']='https://xchfeathrtest4spark.dev.azuresynapse.net'
os.environ['SYNAPSE_POOL_NAME']='spark31'
os.environ['SYNAPSE_WORKSPACE_DIR']='abfss://xchfeathrtest4fs@xchfeathrtest4sto.dfs.core.windows.net/'
os.environ['REDIS_PASSWORD']='YOUR_REDIS_PASSWORD'
os.environ['REDIS_HOST']='xchfeathrtest4redis.redis.cache.windows.net'
os.environ['FEATHR_RUNTIME_LOCATION']='https://azurefeathrstorage.blob.core.windows.net/public/feathr_20220204.jar'
os.environ['AZURE_PURVIEW_NAME']='xchfeathrtest4purview'
os.environ['ADLS_ACCOUNT']='xchfeathrtest4sto'
os.environ['ADLS_KEY']='YOUR_ADLS_KEY'
os.environ['BLOB_ACCOUNT']='xchfeathrtest4sto'
os.environ['BLOB_KEY']='YOUR_BLOB_KEY'
os.environ['SPARK_CONFIG__DATABRICKS__WORKSPACE_INSTANCE_URL']='https://adb-1948202983662686.6.azuredatabricks.net'
os.environ['DATABRICKS_WORKSPACE_TOKEN_VALUE']='YOUR_DATABRICKS_WORKSPACE_TOKEN'
os.environ['JDBC_TABLE']=''
os.environ['JDBC_USER']=''
os.environ['JDBC_PASSWORD']=''
os.environ['JDBC_DRIVER']=''
os.environ['JDBC_AUTH_FLAG']=''
os.environ['JDBC_TOKEN']='' 
os.environ['JDBC_SF_URL']=''
os.environ['JDBC_SF_USER']=''
os.environ['JDBC_SF_ROLE']=''
os.environ['JDBC_SF_PASSWORD']=''

os.environ['FEATURE_REGISTRY__ENDPOINT']='http://localhost:8000/api'

c=FeathrClient("/Users/chenxu/repos/feathrs/test-script/feathr_config.yaml")
p1=c.new_project("p1")

In [2]:
batch_source = p1.hdfs_source(name="nycTaxiBatchSource",
                              path="wasbs://public@azurefeathrstorage.blob.core.windows.net/sample_data/green_tripdata_2020-04.csv",
                              timestamp_column="lpep_dropoff_datetime",
                              timestamp_column_format="yyyy-MM-dd HH:mm:ss")
print("Source id: ", batch_source.id)
request_features = p1.anchor_group("request_features",
                                   source=p1.input_context)
f_trip_distance = request_features.anchor_feature("f_trip_distance",
                                          FeatureType.FLOAT,
                                          transform="trip_distance")
f_trip_time_duration = request_features.anchor_feature("f_trip_time_duration",
                                               FeatureType.INT32,
                                               transform="(to_unix_timestamp(lpep_dropoff_datetime) - to_unix_timestamp(lpep_pickup_datetime))/60")
f_is_long_trip_distance = request_features.anchor_feature("f_is_long_trip_distance",
                                                  FeatureType.BOOLEAN,
                                                  transform="cast_float(trip_distance)>30")
f_day_of_week = request_features.anchor_feature("f_day_of_week",
                                        FeatureType.INT32,
                                        transform="dayofweek(lpep_dropoff_datetime)")

location_id = TypedKey("DOLocationID",
                       ValueType.INT32,
                       full_name="",
                       description="location id in NYC")

agg_features = p1.anchor_group("aggregationFeatures", source=batch_source)

trans_avg = Transformation.window_agg("cast_float(fare_amount)",
                                      Aggregation.AVG,
                                      "90d")
trans_max = Transformation.window_agg("cast_float(fare_amount)",
                                      Aggregation.MAX,
                                      "90d")
f_location_avg_fare = agg_features.anchor_feature("f_location_avg_fare",
                                          FeatureType.FLOAT,
                                          transform=trans_avg,
                                          keys=[location_id])
f_location_max_fare = agg_features.anchor_feature("f_location_max_fare",
                                          FeatureType.FLOAT,
                                          transform=trans_max,
                                          keys=[location_id])

f_trip_time_distance = p1.derived_feature("f_trip_time_distance",
                                  FeatureType.FLOAT,
                                  inputs=[f_trip_distance,
                                          f_trip_time_duration],
                                  transform="f_trip_distance * f_trip_time_duration")
f_trip_time_rounded = p1.derived_feature("f_trip_time_rounded",
                                 FeatureType.INT32,
                                 inputs=[f_trip_time_duration],
                                 transform="f_trip_time_duration % 10")


Source id:  1d7182d9-3764-4fe3-bd22-80bf7f0c622e


In [ ]:
f_trip_time_rounded = p1.derived_feature("f_trip_time_rounded",
                                 FeatureType.INT32,
                                 inputs=[f_trip_time_duration],
                                 transform="f_trip_time_duration % 10")


In [ ]:
anchor_query = FeatureQuery(
    [
        f_trip_distance,
        f_trip_time_duration,
        f_is_long_trip_distance,
        f_day_of_week,
        f_location_avg_fare,
        f_location_max_fare,
    ],
    [location_id],
)

derived_query = FeatureQuery(
    [
        f_trip_time_distance,
        f_trip_time_rounded,
    ],
    [location_id],
)

ob = ObservationSettings("wasbs://public@azurefeathrstorage.blob.core.windows.net/sample_data/green_tripdata_2020-04.csv",
                         "lpep_dropoff_datetime",
                         "yyyy-MM-dd HH:mm:ss")

output = c.get_remote_url("output.bin")

job_id = p1.get_offline_features(
    ob, feature_query=[anchor_query, derived_query], output=output)

message = c.wait_for_job(job_id)
print(message)


In [ ]:
from datetime import datetime

start = datetime(2020, 5, 20, 0, 0, 0)
end = datetime(2020, 5, 22, 0, 0, 0)
sink = RedisSink("table1")
job_ids = p1.materialize_features([f_location_avg_fare, f_location_max_fare],
                        start,
                        end,
                        DateTimeResolution.Daily,
                        sink)
messages = c.wait_for_jobs(job_ids)
print(messages)
